In [1]:
import jsonlines 
import pandas as pd
from collections import Counter, defaultdict

In [2]:
d_anns = []
with jsonlines.open("/home/vs428/Downloads/n2c2_SO_annotation-dc2249.jsonl") as reader:
    d_anns = [line for line in reader]
    
a_anns = []
with jsonlines.open("/home/vs428/Downloads/n2c2_SO_annotation-asg64.jsonl") as reader:
    a_anns = [line for line in reader]    
    
gold_anns = []
with jsonlines.open("/home/vs428/Downloads/n2c2_SO_annotation_GOLD.jsonl") as reader:
    gold_anns = [line for line in reader]        

In [3]:
len(d_anns), len(a_anns), len(gold_anns)

(192, 142, 126)

In [4]:
d_hashes = set([ann['_input_hash'] for ann in d_anns])
a_hashes = set([ann['_input_hash'] for ann in a_anns])
gold_hashes = set([ann['_input_hash'] for ann in gold_anns])

In [5]:
len(d_hashes), len(a_hashes), len(gold_hashes)

(192, 142, 126)

In [6]:
non_gold_hashes = d_hashes.intersection(a_hashes) - gold_hashes

In [7]:
len(non_gold_hashes)

16

In [8]:
non_gold_anns_by_hash = defaultdict(list)
for ann in a_anns:
    if ann['_input_hash'] in non_gold_hashes:
        non_gold_anns_by_hash[ann['_input_hash']].append(ann)
for ann in d_anns:
    if ann['_input_hash'] in non_gold_hashes:    
        non_gold_anns_by_hash[ann['_input_hash']].append(ann)    

In [9]:
non_gold_anns = {hash_term:anns for hash_term, anns in non_gold_anns_by_hash.items() if len(anns) == 2}

In [ ]:
agreed_non_gold_anns = []

In [ ]:
counts = Counter()

for hash_val, anns in non_gold_anns.items():
    if anns[0]['spans'] == anns[1]['spans']:
        counts['overlap'] += 1
        agreed_non_gold_anns.append(anns[0])
    else:
        # print(anns[0]['spans'])
        # print(anns[1]['spans'])  
        # print("\n\n-------\n\n")
        counts['no_overlap'] += 1

In [ ]:
final_training_anns = gold_anns + agreed_non_gold_anns

In [ ]:
f"total number of anns: {len(final_training_anns)}"

In [ ]:
with jsonlines.open("/home/vs428/Downloads/n2c2_SO_annotation_GOLD_MERGED.jsonl", "w") as writer:
    writer.write_all(final_training_anns)


In [ ]:
from transformers import trainer_utils